In [223]:
import pandas as pd

In [224]:
path_global_eco_1 = "../../data/global_economy/year_df_part1.pkl"
path_global_eco_2 = "../../data/global_economy/year_df_part2.pkl"
path_steel_data = "../../data/world_steel_association/Steel_Data_Data_Mining.xlsx"
path_trade_data = "../../data/un_comtrade/trade_data_all_hs.xlsx"

path_output_data = "../../data/dim/"

In [225]:
trade_data = pd.read_excel(path_trade_data)

trade_cols = ["reporterISO", "reporterDesc"]

trade_data = trade_data[trade_cols]

trade_data = trade_data.rename(columns={trade_cols[0]: "country_code", trade_cols[1]: "country_name"})

df_trade = trade_data.drop_duplicates(subset=["country_code", "country_name"])

In [226]:
steel_data = pd.read_excel(path_steel_data)
steel_data.head()


change_cols = ["Country Code", "Country"]

steel_data = steel_data[change_cols]

steel_data = steel_data.rename(columns={change_cols[0]: "country_code", change_cols[1]: "country_name"})

df_steel = steel_data.drop_duplicates(subset=['country_code', 'country_name'])


In [227]:
global_economy_data_1 = pd.read_pickle("../../data/global_economy/year_df_part1.pkl")
global_economy_data_2 = pd.read_pickle("../../data/global_economy/year_df_part2.pkl")

df_global_economy = pd.concat([global_economy_data_1, global_economy_data_2], ignore_index=True)

change_cols = ["Code", "Country", "ContinentCode"]

df_global_economy = df_global_economy[change_cols]

df_global_economy = df_global_economy.rename(columns={change_cols[0]: "country_code", change_cols[1]: "country_name"})

df_global_economy = df_global_economy.drop_duplicates(subset=['country_code', 'country_name'])


In [228]:
north_america_countries = [
    'Antigua and Barbuda', 'Aruba', 'Bahamas', 'Barbados', 'Belize', 'Bermuda',
    'Canada', 'Costa Rica', 'Dominica', 'Dominican Republic', 'El Salvador',
    'Grenada', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico',
    'Nicaragua', 'Panama', 'Puerto Rico', 'Saint Lucia',
    'Saint Vincent and the Grenadines', 'Trinidad and Tobago',
    'USA', 'Cuba', 'Montserrat'
]

# Ergänze ggf. die fehlenden ContinentCodes in deinem DataFrame
df_global_economy.loc[
    df_global_economy['country_name'].isin(north_america_countries),
    'ContinentCode'
] = 'NA'

# Index (ContinentCode) in neue Spalte übernehmen
df_global_economy = df_global_economy.reset_index().drop(columns='index')


# Optional: Spaltenreihenfolge anpassen
cols = ['ContinentCode'] + [col for col in df_global_economy.columns if col not in ['ContinentCode']]
df_global_economy = df_global_economy[cols]

In [229]:
df_global_economy.describe()

,ContinentCode,country_code,country_name
count,202,202,202
unique,6,202,202
top,AF,AFG,Afghanistan
freq,53,1,1


In [230]:
df_steel.describe()

,country_code,country_name
count,121,121
unique,121,121
top,ALB,Albania
freq,1,1


In [231]:
df_trade.describe()

,country_code,country_name
count,169,169
unique,169,169
top,ALB,Albania
freq,1,1


In [232]:
df_dim_country = df_global_economy.merge(df_steel, on=['country_code'], how='outer')
df_dim_country = df_dim_country.merge(df_trade, on=['country_code'], how='outer')

df_dim_country["country_name"] = (
    df_dim_country["country_name_x"]
    .fillna(df_dim_country["country_name"])
    .fillna(df_dim_country["country_name_y"])
)
df_dim_country = df_dim_country.drop(columns=["country_name_x", "country_name_y"])

df_dim_country.describe()

,ContinentCode,country_code,country_name
count,202,210,210
unique,6,210,209
top,AF,ABW,Costa Rica
freq,53,1,2


In [233]:
df_dim_country_na = df_dim_country[
    df_dim_country["country_name"].duplicated(keep=False)
]

df_dim_country_na.head()

,ContinentCode,country_code,country_name
42,NA,CRC,Costa Rica
43,NaN,CRI,Costa Rica


In [234]:
# adjust if needed 
list_to_del = ["CRI", "WLD"]
df_dim_country = df_dim_country[~df_dim_country["country_code"].isin(list_to_del)]
df_dim_country.describe()

,ContinentCode,country_code,country_name
count,202,208,208
unique,6,208,208
top,AF,ABW,Aruba
freq,53,1,1


In [235]:
df_dim_country_con = df_dim_country[df_dim_country["ContinentCode"].isna()]

df_dim_country_con.head(20)

,ContinentCode,country_code,country_name
45,NaN,CUW,Curaçao
46,NaN,CYM,Cayman Isds
155,NaN,PSE,State of Palestine
156,NaN,PYF,French Polynesia
161,NaN,S19,"Other Asia, nes"
185,NaN,TLS,Timor-Leste


In [236]:
# Mapping Dictionary: ISO-Code → Englischer Name
continent_name_map = {
    'AF': 'Africa',
    'AS': 'Asia',
    'EU': 'Europe',
    'NA': 'North America',
    'SA': 'South America',
    'OC': 'Oceania',
    'AN': 'Antarctica',
    'RW': 'Rest of World',
}
# Englische Kontinentnamen hinzufügen
df_dim_country['ContinentCode'] = df_dim_country['ContinentCode'].fillna('RW')

df_dim_country['Continent'] = df_dim_country['ContinentCode'].map(continent_name_map)

df_dim_country.to_csv(path_output_data + "dim_country.csv")